In [17]:

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
from ase.io.extxyz import read_xyz
import numpy as np
from ase.atoms import Atoms
from ase.io.formats import index2range
from ase.io.cfg import read_cfg
from io import StringIO, UnsupportedOperation
from mtp import *
import os
from matplotlib import pyplot as plt
from quippy.potential import Potential
import pickle
from Ge_analysis import *
from Ge_calculation import *
import matplotlib.pyplot as plt
from matscipy.rings import ring_statistics

In [13]:
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = [ 'Helvetica', 'Arial', 'FreeSans', 'DejaVu Sans' ]

plt.rcParams['axes.labelsize'] = 18
plt.rcParams['xtick.labelsize'] = 15
plt.rcParams['ytick.labelsize'] = 15

plt.rcParams['legend.fontsize'] = 14
plt.rcParams['legend.title_fontsize'] = 14
plt.rcParams['legend.fancybox'] = False
plt.rcParams['legend.edgecolor'] = 'k'
plt.rcParams['legend.borderaxespad'] = 1.5

plt.rcParams['lines.linewidth'] = 3.0
plt.rcParams['axes.linewidth'] = 1.5

plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = 10
plt.rcParams['xtick.major.width'] = 1.0
plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['xtick.minor.width'] = 1.0

plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['ytick.major.size'] = 10
plt.rcParams['ytick.major.width'] = 1.0
plt.rcParams['ytick.minor.size'] = 5
plt.rcParams['ytick.minor.width'] = 1.0

# Labelling

In [3]:
GAP_18_dir = '/home/joe/Documents/Ge_od/Si_GAP_training/libAtoms-silicon-testing-framework-fc252cb/models/GAP'
with open(GAP_18_dir + '/gp_iter6_sparse9k.xml.xyz', 'r') as f:
    GAP_18_db = list(read_xyz(f, index=slice(0,None)))
GAP_18_pot = Potential(param_filename=GAP_18_dir+'/gp_iter6_sparse9k.xml')

/home/joe/ase/ase/calculators/calculator.py:507: FutureWarning: The keyword "ignore_bad_restart_file" is deprecated and will be removed in a future version of ASE.  Passing more than one positional argument to Calculator is also deprecated and will stop functioning in the future.  Please pass arguments by keyword (key=value) except optionally the "restart" keyword.
  warnings.warn(FutureWarning(


In [21]:
for i, val in enumerate(GAP_18_db):

    GAP_18_pot.calculate(val, properties=['energies', 'forces', 'stress'])
    val.info['gap18_energy'] = GAP_18_pot.results['energy']
    val.info['gap18_virial'] = GAP_18_pot.results['virial']
    val.arrays['gap18_force'] = GAP_18_pot.results['forces']

In [22]:
GAP_18_db[-1].info

{'config_type': 'st12',
 'Minim_Constant_Volume': True,
 'dft_virial': array([24.95940288, -3.22408819,  7.76967253, -3.22408819,  6.25097899,
        -4.20318319,  7.76967253, -4.20318319, 17.61407929]),
 'dft_energy': -1952.46351935,
 'cutoff': -1.0,
 'nneightol': 1.2,
 'energy': -1952.4764130029887,
 'stress': array([-0.1209391 , -0.03121241, -0.08841763,  0.01958532, -0.03768385,
         0.01490357]),
 'gap18_energy': -1952.4764130029887,
 'gap18_virial': array([[24.73930912, -3.04867443,  7.70861006],
        [-3.04867443,  6.38481257, -4.00637357],
        [ 7.70861006, -4.00637357, 18.08671514]])}

In [7]:
# GAP_18_pot.calculate(GAP_18_db[0], properties=['energies', 'forces', 'stress'])
GAP_18_pot.results

{}

In [27]:
for i in GAP_18_db:
    i.set_calculator(None)

In [4]:
os.chdir('/home/joe/Documents/Ge_od/GAP_to_MTP')


In [28]:
with open('GAP_18_db_GAPlabelled.pickle', 'wb') as f:
    pickle.dump(GAP_18_db, f)

In [6]:
with open('GAP_18_db_GAPlabelled.pickle', 'rb') as f:
    GAP_18_db = pickle.load(f)

In [5]:
with open('GAP_18_db_GAPlabelled.cfg', 'w') as f:
    write_cfg_db(f, GAP_18_db, energy_name='gap18_energy', force_name='gap18_force', virial_name='gap18_virial')

KeyError: 'gap18_force'

In [25]:
with open('GAP_18_db_DFTlabelled.cfg', 'w') as f:
    write_cfg_db(f, GAP_18_db, energy_name='dft_energy', force_name='dft_force', virial_name='dft_virial')

In [24]:
GAP_18_db[10].info['dft_virial']

array([ 0.135393 , -0.41881  ,  0.59576  , -0.41881  ,  0.805479 ,
       -0.0964051,  0.59576  , -0.0964051,  0.969778 ])

## Writing all melt-quench data to DB for fitting


In [3]:
len(mq_db)

NameError: name 'mq_db' is not defined

In [41]:
sb = '/home/joe/Documents/Ge_od/Structure_databases'
mq_dirs = [os.path.join(sb, i) for i in ['run_64_atoms', 'run_125_atoms', 'run_216_atoms']]

mq_db = []

for i in mq_dirs:
    for j in os.listdir(i):
        if 'run_' in j and 'castep' not in j:
            for k in os.listdir(os.path.join(i, j, 'NPT', )):
                at = read_cfg(os.path.join(i,j,'NPT',k))
                at.info['rundir'] = j
                mq_db.append(at)

l = len(mq_db)
for i, val in enumerate(mq_db):

    GAP_18_pot.calculate(val, properties=['energies', 'forces', 'stress'])
    val.info['gap18_energy'] = GAP_18_pot.results['energy']
    val.info['gap18_virial'] = GAP_18_pot.results['virial']
    val.arrays['gap18_force'] = GAP_18_pot.results['forces']
    if (i//l * 100) % 10 == 0:
        print('{}% done'.format(i//l *100), datetime.now().time())

with open('mq_db_GAP-18labelled.pickle', 'wb') as f:
    pickle.dump(mq_db, f)

# Validation of indirect MTP
compare as far as possible to original tests on GAP-18 from paper
* ring stats
* structure factors
* vacancy formation energies

In [ ]:
GAP_18_md =

In [1]:

MTP_dir = '/home/joe/Documents/Ge_od/GAP_to_MTP/MTPs/'

In [ ]:

MTP = [MTP(os.path.join(MTP_dir, 'u16_min1.5/GAP_18_db_GAPlabelled.mtp'), potential_name='u16_m1.5')]
MTP_md = [MD_run(os.path.join(MTP_dir, 'u16_min1.5/run_1000_'), label='u16_m1.5')]

In [ ]:

rs = [ring_statistics(i.df['Configs'][240000], cutoff=2.7) for i in MTP_md]
rs = pad_rstats(rs)

In [ ]:
fig, axs = plt.subplots()
# colormap = plt.get_cmap('viridis')
# colors = [colormap(i) for i in np.linspace(F_reg[0], F_reg[-1], len(F_reg))]
for i, val in enumerate(rs):
axs.plot(np.arange(2, len(rs[0])), val[2:], label=MTP_md[i].label,
color=colors[i], marker='s')
axs.set(xlim=(2,11), ylim=(-5,5))
axs.legend()
axs.set(xlabel='Ring size', ylabel='Count')
axs.tick_params(axis="both",direction="in", left="off",labelleft="on")
plt.tight_layout()

In [44]:
from datetime import datetime